In [1]:
from spacy import displacy
import en_core_web_sm
nlp = en_core_web_sm.load()

import nltk
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [153]:
class Person():
    def __init__(self, name):
        self.name = name
        self.attr = {}
        self.verb = {}
        self.bag = {}
entities = []

In [171]:
def process(inp):
    if '?' in inp:
        question(inp)
    else:
        statement(inp)

def decompose(doc):
    root = [token for token in doc if token.head == token][0]
    subject = list(root.lefts)[0]
    flag_not = True
    attr = None
    for tok in root.subtree:
        if tok.dep_ == 'neg':
            flag_not = False
        if tok.dep_ == 'attr' or tok.dep_ == 'acomp' or tok.dep_ == 'dobj' or tok.dep_ == 'prep':
            attr = tok
    return root, subject, attr, flag_not

def proc_attr(attr):
    return " ".join([tok.lemma_ for tok in attr if tok.text != 'a' and tok.text != 'the'])
    

def statement(inp):
    doc = nlp(inp)
    root, subj, atr,flag_not = decompose(doc)
    obj = get_entity(str(subj.text))

        
    if 'be' == root.lemma_:
        atr = proc_attr(doc[atr.left_edge.i : atr.right_edge.i+1])
        obj.attr[atr] = flag_not
    elif 'have' == root.lemma_:
        atr = proc_attr(doc[atr.left_edge.i : atr.right_edge.i+1])
        obj.bag[atr] = flag_not
    else:
        atr = proc_attr(doc[atr.left_edge.i : atr.right_edge.i+1])
        if flag_not:
            obj.verb.setdefault(root.lemma_, set([])).add(atr)
        else:
            obj.verb[root.lemma_].discard(atr)
    for tok in subj.subtree:
        if tok.dep_ == 'compound':
            obj.attr[tok.lemma_.lower()] = True

    
def get_entity(name):
    for i in entities:
        if i.name == name:
            return i
    ret = Person(name)
    entities.append(ret)
    return ret

def decompose_question(doc):
    root = [token for token in doc if token.head == token][0]
    subject = [token for token in root.subtree if token.dep_=='nsubj']
    subject = subject[0] if len(subject)>0 else None
    flag_not = True
    attr = None

    for tok in root.subtree:
        if tok.dep_ == 'neg':
            flag_not = False
        if tok.dep_ == 'attr' or tok.dep_ == 'acomp' or tok.dep_ == 'dobj' or tok.dep_ == 'prep':
            attr = tok
            if subject is None:
                subject = [token for token in tok.subtree if token.dep_=='compound'][0]
            if subject.lemma_=='who':
                subject = None
    return root, subject, attr, flag_not

def question(inp):
    doc = nlp(inp)

    root, subj, atr,flag_not = decompose_question(doc)
    atr = proc_attr(doc[atr.left_edge.i : atr.right_edge.i+1])
    if 'who' in inp.lower():
        if 'be' == root.lemma_:
            print(", ".join([ent.name for ent in entities if ent.attr.get(atr, None) is not None and ent.attr.get(atr, None)==flag_not]))
        elif 'have' == root.lemma_:
            print(", ".join([ent.name for ent in entities if ent.bag.get(atr, False) == flag_not]))
        else:
            print(", ".join([ent.name for ent in entities if atr in ent.verb.get(root.lemma_, [])]))
    elif 'what' in inp.lower():
        obj = get_entity(str(subj.text))
        if 'have' == root.lemma_:
            print(", ".join([elem for (elem,val) in obj.bag.items() if val]))
        else:
            print(obj.verb.get(root.lemma_, []))
            
    else:
        obj = get_entity(str(subj.text))
        if 'be' == root.lemma_:
            print(obj.attr.get(atr, "Unknown"))
        elif 'ha' == root.lemma_:
            print(obj.bag.get(atr, "Unknown"))
        else:
            print(obj.verb.get(root.lemma_, "Unknown"))

a = input("> ")
while a:
    process(a)
    a = input("> ")

> What does Stas have?
bicycle, byicle
> What does Stas eat?
['pizza']
> Who is old?
Stas, Giancarlo
> Who is not old?

> Who has a bicycle>
> Who has a bicycle?
Stas, Alena, Who
> Who does not have a bicycle?
Giancarlo, Marina
> 


In [163]:
for ent in entities:
    print(ent.__dict__)

{'name': 'Stas', 'attr': {'old': True, 'doctor': False, 'red': True}, 'verb': {'live': ['in Innopolis'], 'eat': ['pizza']}, 'bag': {'bicycle': True, 'byicle': True}}
{'name': 'Giancarlo', 'attr': {'old': True}, 'verb': {'live': ['in Zion']}, 'bag': {}}
{'name': 'Alena', 'attr': {}, 'verb': {}, 'bag': {'bicycle': True}}
{'name': 'Marina', 'attr': {'doctor': True}, 'verb': {}, 'bag': {}}


In [21]:
displacy.render(nlp(input()), jupyter=True, style='dep')

Vadim has 3 dildos in ass.
